## Name: Karma Tarap
### CSCI E-89C Deep Reinforcement Learning  
### Part II of Assignment 4

## Problem 1 (10 points)

Consider Environment that has five states: 1, 2, 3, 4, and 5. Possible transitions are: (1) 1->1, 1->2; (2) 2->1, 2->2, 2->3; (3) 3->2, 3->3, 3->4; (4) 4->3, 4->4, 4->5; (5) 5->4, 5->5.

Actions of the Agent are decoded by -1, 0, and +1, which correspond to its intention to move left, stay, and move right, respectively. The Environment, however, does not always respond to these intentions exactly, and there is 10% chance that action 0 will result in moving to the left (if moving to the left is admissible), and +1 action will result in staying - in other words, there is an "east wind." More specifically, the non-zero transition probabilities $p(s^\prime,r|s,a)$ are<br>

$p(s^\prime=1,r=0|s=1,a=0)=1$,<br>
$p(s^\prime=1,r=0|s=1,a=+1)=0.1,p(s^\prime=2,r=0|s=1,a=+1)=0.9$,<br>

$p(s^\prime=1,r=0|s=2,a=-1)=1$,<br>
$p(s^\prime=1,r=0|s=2,a=0)=0.1,p(s^\prime=2,r=0|s=2,a=0)=0.9$,<br>
$p(s^\prime=2,r=0|s=2,a=+1)=0.1,p(s^\prime=3,r=1|s=2,a=+1)=0.9$,<br>

$p(s^\prime=2,r=0|s=3,a=-1)=1$,<br>
$p(s^\prime=2,r=0|s=3,a=0)=0.1,p(s^\prime=3,r=1|s=3,a=0)=0.9$,<br>
$p(s^\prime=3,r=1|s=3,a=+1)=0.1,p(s^\prime=4,r=0|s=3,a=+1)=0.9$,<br>

etc.

Further, we assume that whenever the process enters state 3, the Environment generates reward = 1. In all other cases the reward is 0. For example, transition 2->3 will result in reward 1, transition 3->3 will result in reward 1, transition 3->2 will result in reward 0, transition 2->2 will result in reward 0, etc.



Further, assume that the agent does not know about the wind or what rewards to expect. It chooses to stay in all states, i.e. the policy is
$\pi(-1|1)=0, \pi(0|1)=1, \pi(+1|1)=0$,<br>
$\pi(-1|2)=0, \pi(0|2)=1, \pi(+1|2)=0$,<br>
$\pi(-1|3)=0, \pi(0|3)=1, \pi(+1|3)=0$,<br>
$\pi(-1|4)=0, \pi(0|4)=1, \pi(+1|4)=0$,<br>
etc.

Please estimate the action-value function using First-Visit Monte Carlo (MC) prediction. Make sure each pair $(S_0,A_0)$ has non-zero probability of being selected. Let’s use $\gamma=0.9$ and run the episodes for $T=100$. Make some reasonable selection of tolerance $\theta$.

Based on the estimates of $q_\pi(s,a)$, please suggest a better policy. Is it optimal?

In [232]:
import random
from matplotlib import pyplot as plt 
import numpy as np

class Environment:
    def __init__(self, S0 = 1, A0 = 1):
        self.time = 0
        self.state = S0
        self.action = A0

    def admissible_actions(self):
        A = list((-1,0,1))
        if self.state == 1: A.remove(-1)
        if self.state == 5: A.remove(1)
        return A
    
    def check_state(self):
        return self.state

    def get_reward(self, action):
        self.time += 1
        move = action
        if self.state > 1 and move > -1:
            move = np.random.choice([move-1, move],p=[0.1,0.9])
        self.state += move
        self.action = move
        if self.state == 3:
            reward = 1
        else:
            reward = 0
        return reward

In [233]:
class Agent:
    def __init__(self):
        self.current_reward = 0.0

    def step(self, env):
        #actions = env.admissible_actions()
        action_selected = 0
        reward = env.get_reward(action_selected)            
        self.current_reward = reward

In [250]:
def generate_episode(T, env, agent):
    episode = []
    for i in range(T):
        agent.step(env)
        episode.append((env.state, env.action, agent.current_reward))
    return episode

def first_visit_mc(episodes = 10000, gamma=0.9):
    R = np.zeros(5)
    N = np.zeros(5)
    T = 100
    for _ in range(episodes):
        env = Environment(random.choice(range(1,6)), random.choice([-1,0,1]))
        agent = Agent()
        episode = generate_episode(T, env, agent)
        
        states = [e[0] for e in episode]
        rewards = [e[2] for e in episode]

        #print("states:",states)
        #print("rewards:",rewards)
        G = 0
        for j in range(T-1,-1,-1):
            state = states[j]
            G += rewards[j]*gamma
            if state not in states[0:j]:
                R[state-1] += G
                N[state-1] += 1

    return R / N

first_visit_mc()

array([0.  , 0.  , 8.83, 8.84, 8.69])

array([0.  , 0.  , 8.95, 8.85, 8.81])

## Problem 2 (10 points)

In this problem, the agent will obtain the optimal policy via MC control. Please run the On-policy first-visit MC control (for $\varepsilon$-soft policies)  without Exploring Starts algorithm for estimating $\pi_*$. Use same $\gamma=0.9$ and $T=100$.

Does the final policy appear to be optimal?